In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import function_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

loc = "angular_distr_el/CH9/ID_ALL_mol_e0_valid/EN_gate/MFPADs_multinew_std"

fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
k=fileRCR[loc].keys()
d=dict(fileRCR[loc].classes()) # shows the type of each item
dict(fileRCR[loc]) # keys : values
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
# type(check)
# check._members()

In [ ]:
MFPAD_RCR=[] # initilialises a list
ctetha_RCR=[]
cosphi=[]
MFPAD_xy=[]
run_one=0
for key, value in fileRCR[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            # print(filename)
            ctheta_nm=float((str(key).split("costheta_")[1]).split("_phi")[0])
            phi_nm=float((str(key).split("phi_")[1]).split(";")[0])
            # phicos.append((ctheta_nm,phi_nm))
            cosphi.append((ctheta_nm+0.1666,phi_nm+15)) # centered in the middle of the bins
            #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1) #alternative with import re
            temp=np.array(fileRCR[filename].numpy())
            MFPAD_RCR.append(temp[0]) # it is a list!
            if run_one == 0:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_one=1
        elif "cos(theta)" in filename.lower():
            ctheta_nc=float((str(key).split("costheta_")[1]).split("_phi")[0])
            temp=np.array(fileRCR[filename].numpy())
            ctetha_RCR.append(temp[0]) # it is a list!
            ctheta_c = temp[1] # cos(theta) from 1D
MFPAD_RCR=np.array(MFPAD_RCR)
ctetha_RCR=np.array(ctetha_RCR) # it is a list!

In [ ]:
MFPAD_RCL=[] # initilialises a list
ctetha_RCL=[]
cosphi=[]
MFPAD_xy=[]
run_one=0
for key, value in fileRCL[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            # print(filename)
            ctheta_nm=float((str(key).split("costheta_")[1]).split("_phi")[0])
            phi_nm=float((str(key).split("phi_")[1]).split(";")[0])
            # phicos.append((ctheta_nm,phi_nm)) #they are inverted
            cosphi.append((ctheta_nm+0.1666,phi_nm+15))  # centered in the middle of the bins
            #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1) #alternative with import re
            temp=np.array(fileRCL[filename].numpy())
            MFPAD_RCL.append(temp[0]) # it is a list!
            if run_one == 0:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_one=1
        elif "cos(theta)" in filename.lower():
            ctheta_nc=float((str(key).split("costheta_")[1]).split("_phi")[0])
            temp=np.array(fileRCL[filename].numpy())
            ctetha_RCL.append(temp[0]) # it is a list!
            ctheta_c = temp[1] # cos(theta) from 1D
MFPAD_RCL=np.array(MFPAD_RCL)
ctetha_RCL=np.array(ctetha_RCL) # it is a list!

In [ ]:
cosn=np.array([col[0] for col in cosphi]); #list
phin=np.array([col[1] for col in cosphi]); #list
cosM=(MFPAD_xy[0][1][1:] + MFPAD_xy[0][1][:-1])/2
phiM=(MFPAD_xy[0][0][1:] + MFPAD_xy[0][0][:-1])/2

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20, 6.5),sharex='col')
fig.subplots_adjust(hspace = .5, wspace=.5)

ax[0].contourf(phin.reshape(12,6),cosn.reshape(12,6),MFPAD_RCL[0])
ax[1].contourf(phiM,cosM,MFPAD_RCL[0].T) #has to be transposed becasue how it has been defined in lmf2root
ax[2].contourf(MFPAD_RCL[0]) #this too

fig.tight_layout()

MFPAD_RCL[0].shape

In [ ]:
import itertools
#cartesian product
phicos_MFPAD = np.array(list(itertools.product(phiM, cosM)))
cosphi_MFPAD = np.array(list(itertools.product(cosM, phiM)))
# print(phicos_MFPAD,cosphi_MFPAD)
theta_rad = np.arccos([col[0] for col in cosphi_MFPAD])
phi_rad = np.array([col[1] for col in cosphi_MFPAD]) * np.pi/180.
counts=MFPAD_RCR[0].reshape(-1)
x = counts * np.sin(theta_rad) * np.cos(phi_rad)
X = x.reshape(12,6)
y = counts * np.sin(theta_rad) * np.sin(phi_rad)
Y = y.reshape(12,6)
z = counts * np.cos(theta_rad)
Z = z.reshape(12,6)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

In [ ]:
# Go scatters do not provide a legend
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=d,            # set color to an array/list of desired values
                        colorscale='Viridis',   # choose a colorscale
                        opacity=1.,
                        showscale=True          # to show the legend according to the color
                   )
                )])

fig.update_layout(title="TFMeOx MFPADs theory scattered",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
import triangulation as tr
from scipy.spatial import Delaunay
points2d_trace = go.Scatter(x=x, y=y, mode='markers', marker_color='red', marker_size=6)
point_trace= go.Scatter(x=x, y=y, 
                         mode='markers',
                         name='points',
                         marker_color='red',
                         marker_size=6)
pts2d = np.array([x,y]).T
tri = Delaunay(pts2d)
delaunay_tri = tr.triangulation_edges(pts2d, tri.simplices, linewidth=1)
fig2 = go.Figure(data=[delaunay_tri, point_trace])
fig2.update_layout(width=500, height=500);
fig2.show()

i, j, k = tri.simplices.T
my_mesh3d = go.Mesh3d(
    x = x,
    y = y,
    z = z,
    i=i, j=j, k=k,
    colorscale='deep_r',
    colorbar_thickness=25,
    intensity=z, 
    flatshading=True
)

my_layout = dict(width=800, height=600,
                 scene_camera_eye=dict(x=1.5, y=1.5, z=1), 
                 scene_aspectratio=dict(x=1.5, y=1.5, z=1))

points3d = np.array([x,y,z]).T

delaun_tri3d  = tr.triangulation_edges(points3d, tri.simplices)

fig3d = go.Figure(data=[my_mesh3d, delaun_tri3d])

fig3d.show()

In [ ]:
from pyntcloud import PyntCloud, structures
temp = pd.DataFrame(np.hstack((x[:,None], y[:,None], z[:,None])))
temp.columns = ["x", "y", "z"]
sph = PyntCloud(temp)
x,  y,  z = sph.points["x"], sph.points["y"], sph.points["z"]
pts = sph.points[['x', 'y', 'z']]
delaun = structures.Delaunay3D(pts)

delaun.compute()
mesh = delaun.get_mesh()
tri= mesh[['v1', 'v2', 'v3']].values
I, J, K = tri.T

fig1 = go.Figure(go.Mesh3d(x=x, y=y, z=z, 
                           i=I, j=J, k=K, 
                           intensity=z,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,
                                         diffuse=1,
                                         fresnel=4,        
                                         specular=0.5,
                                         roughness=0.05,
                                         facenormalsepsilon=0,
                                         vertexnormalsepsilon=0),
                          lightposition=dict(x=100,
                                             y=100,
                                            z=1000)))
fig1.update_layout(width=800, height=800)
fig1.show()


In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Surface(z=Z, surfacecolor=d_matrix)])
fig.update_layout(title='TFMeOx MFPADs theory surf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()


In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

cosphi=np.array(cosphi)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phin.reshape(12,6),cosn.reshape(12,6),MFPAD_RCL[i])
    
fig.tight_layout()
plt.show()